<a href="https://colab.research.google.com/github/aidanrogers02/aidanrogers02-Album-Popularity-Trends/blob/main/Album_Popularity_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Album Popularity Trends/Mariah Carey’s Album Popularity

**From the Spotify API Documentation**
<br/>
Their definition of popularity:
<br/> 
"The popularity of the track. The value will be between 0 and 100, with 100 being the most popular.
The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are."
<br/>
"Artist and album popularity is derived mathematically from track popularity."

Retrieve Access Token to Make Call to Spotify API

In [2]:
#@title Retrieve Access Token to Make Call to Spotify API
"""An access token is needed to retrieve info from the Spotify API so we are 
getting one"""
import requests


def get_spotify_access_token(filename):
  """Open and read file to get codes Spotify API wants. Return access token"""

  try:
    with open(filename) as f:
      client_id = f.readline()
      client_secret = f.readline()
  except FileNotFoundError:
    print("File does not exist.")

  # Strip white space at the ends
  client_id = client_id.strip()
  client_secret = client_secret.strip()

  # Make post reqeust to Spotify to get access token
  response = requests.post(
    url="https://accounts.spotify.com/api/token",
    data={"grant_type": "client_credentials"},
    auth=(client_id, client_secret),
  )

  # We have the access token
  access_token = response.json()["access_token"]
  return access_token

filename = 'drive/MyDrive/codes/client_codes.txt'

access_token = get_spotify_access_token(filename)

Input Which Artist You Want

In [3]:
#@title Input Which Artist You Want
def make_artist_string():
  print("Input an artist's name.")
  inp = input("Artist Name: ")

  inp = inp.rstrip()
  inp = inp.replace(' ', '%20')
  inp = inp.replace(',', '%2C')

  return inp

def make_artist_url(inp):
  url_start = 'https://api.spotify.com/v1/search?q=artist%3A'
  url_end = '&type=artist&limit=1&offset=0'
  url_whole = ''

  url_whole = (url_start + inp + url_end)

  return url_whole

def retrieve_artist_info(artist_url):
  headers = {"Authorization": f"Bearer {access_token}"}
  artist_response = requests.get(artist_url, headers=headers)

  artist_response_dict = artist_response.json()

  return artist_response_dict

def get_artist_info(single_artist_dict):
  artist_info = {}
  for artist in single_artist_dict:
    artist_info['name'] = artist['name']
    temp_uri = artist['uri']
    artist_info['uri'] = temp_uri[15:]
    artist_info['popularity'] = artist['popularity']
    artist_info['genres'] = artist['genres']
    artist_info['followers'] = artist['followers']['total']

  return artist_info

inp = make_artist_string()
artist_url = make_artist_url(inp)
artist_response_dict = retrieve_artist_info(artist_url)

single_artist_dict = artist_response_dict['artists']['items']

artist_info = get_artist_info(single_artist_dict)

print(f"Artist Found: {artist_info['name']}")

Input an artist's name.
Artist Name: Mariah Carey
Artist Found: Mariah Carey


Get Artist's Albums

In [4]:
#@title Get Artist's Albums: code to allow the "Get Several Albums" to work
"""Get every album from the artist and with them create a url that
can be used to get specific info on all of these albums"""
import requests 

# List of albums/album info that we do not want graphed
restricted_albums = [
    "Me. I Am Mariah…The Elusive Chanteuse (Deluxe)",
    "The Emancipation Of Mimi (Ultra Platinum Edition)",
    "spotify:album:6GhWU8SIWavH4IQAoeJazI",
    "Me. I Am Mariah... The Elusive Chanteuse"
]


def get_album_uris(album_list_dict, restricted_albums):
  
  uris = ""
  # Loop to get and check album info. Create a URL with all of the album uris
  for position, album_list_dict in enumerate(albums_list_dict):
    album_name = album_list_dict['name']
    album_release = album_list_dict['release_date']
    uri = album_list_dict['uri']
    if album_name in restricted_albums or uri in restricted_albums:
      continue
    print(album_name)
    

    # Remove unnecessary prefix
    uri = uri[14:]
    # Add the uri plus the seperator
    uris += uri + "%2C"

    # Spotify API does not like more than 19 albums requested at once
    if position == 19:
      break

  uris = uris[:-3]
  # uris = (uris + "&market=ES")
  uris = ("https://api.spotify.com/v1/albums?ids=" + uris)
  return uris

# URL & headers needed for GET request
url_get_artist_albums = ("https://api.spotify.com/v1/artists/" + artist_info['uri'] + "/albums?include_groups=album&market=ES&limit=50&offset=0")
headers = {"Authorization": f"Bearer {access_token}"}

# Get the list of albums from Spotify
albums_response = requests.get(url_get_artist_albums, headers=headers)
# Format list as a dictionary
albums_response_dict = albums_response.json()

# Print the status code, 200 means it worked. And print the diotionary
print(albums_response.status_code)
# print(albums_response_dict)

# This gives all the albums
albums_list_dict = albums_response_dict['items']


uris = get_album_uris(albums_list_dict, restricted_albums)
print(uris)



200
Mariah Carey's Magical Christmas Special (Apple TV+ Original Soundtrack)
The Rarities
Merry Christmas (Deluxe Anniversary Edition)
Caution
Me. I Am Mariah…The Elusive Chanteuse
Merry Christmas II You
Memoirs of an imperfect Angel (International Version)
E=MC2
The Emancipation of Mimi
Charmbracelet
Glitter
Rainbow
Butterfly
Daydream
Merry Christmas
https://api.spotify.com/v1/albums?ids=5VfesyhwiNgpEEPXlO5c84%2C0v1DRRYBXYg1uVN1CIsyy0%2C0cS9prZ8u3fdbc7lmtCaMV%2C64zK6tmksJw9gNZR0L4DVx%2C6hGRm1piHNE0Xp7I5Guesy%2C4q88opkbXkvvL0iIvbs0pv%2C0an7K6LfvYERpVviYMUFp0%2C31MluXLYC0ZnCSfUZ5T4GX%2C6ek7Y68IlB6CoFkkc2gEQb%2C2UYie72CXeuwIB7YepDidw%2C2hHFZLYnwsYOOxTCrlNvg0%2C1iSTXHBhLc9ImaqyvVZGft%2C7aDBFWp72Pz4NZEtVBANi9%2C1ibYM4abQtSVQFQWvDSo4J%2C61ulfFSmmxMhc2wCdmdMkN


Get Several Albums

In [5]:
#@title Get Several Albums
"""With the URL of all of the artist's albums that we got we now want to make
a plot of the popularity of the albums and the dates they were released"""
import requests 
from datetime import datetime
import plotly.graph_objects as go
from plotly import offline

def plot_album_popularity(albums_dict):
  # Make lists of album info that is to be plotted
  dates, names, popularities, total_tracks_list, labels = [], [], [], [], []
  for album_dict in albums_dict:
    name = album_dict['name']
    popularity = album_dict['popularity']
    date = album_dict['release_date']
    total_tracks = album_dict['total_tracks']

    label = f"{name}: {date}"

    dates.append(date)
    names.append(name)
    popularities.append(popularity)
    total_tracks_list.append(total_tracks)
    labels.append(label)

  # Set up the data to be plotted
  data = [{
    'type': 'scatter',
    'x': dates,
    'y': popularities,
    'hovertext': labels,
    'text': names,
    'textfont_color': 'rgba(250,10,10,1.0)',
    'textposition': 'top right',
    'mode': 'text+markers+lines',
    'marker': {
        'color': 'rgba(193,0,0,1.0)',
        'size': 10.0,
    },
    'line' :{
        'color': 'rgba(133,50,50,1.0)'
    }
  }]
  # Create layout with dictionary to allow more customization
  my_layout = {
    'title': f"Popularity of {artist_info['name']}'s Albums Over the Years",
    'titlefont': {'size': 28},
    'plot_bgcolor': 'rgba(161, 183, 209, 1.0)',
    'xaxis': {
        'title': 'Album Release Year',
        'titlefont': {'size': 24},
        'tickfont': {'size': 14},
    },
    'yaxis': {
        'title': 'Popularity',
        'titlefont': {'size': 24},
        'tickfont': {'size': 14},
    },
  }


  fig = {'data': data, 'layout': my_layout}
  return fig

def plot_artist_info(artist_info):

  names = []
  popularities = []

  names.append(artist_info['name'])
  popularities.append(artist_info['popularity'])

  data = [{
    'type': 'bar',
    'x': names,
    'y': popularities,
    'width': 0.5,
    'marker': {
      'color': 'rgba(193,0,0,1.0)',
    },
  }]
  my_layout = {
      'title': f"{artist_info['name']}'s Popularity",
      'titlefont': {'size': 28},
    'plot_bgcolor': 'rgba(161, 183, 209, 1.0)',
    'xaxis': {
        'title': "Artist's Name",
        'titlefont': {'size': 24},
        'tickfont': {'size': 14},
    },
    'yaxis': {
        'title': 'Popularity',
        'titlefont': {'size': 24},
        'tickfont': {'size': 14},
    },
  }

  fig = {'data': data, 'layout': my_layout}
  return fig



url = uris
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.get(url, headers=headers)
response_dict = response.json()

print(response.status_code)

albums_dict = response_dict['albums']

fig = plot_album_popularity(albums_dict)

fig_art = plot_artist_info(artist_info)

plot = go.Figure(fig)
plot_art = go.Figure(fig_art)

# Plot the graph in the terminal and create an html file of it
offline.plot(fig, filename="popularity_of_mariah.html")
plot.show()
plot_art.show()


200
